#### Author: Roberto Menzoa
#### Date: 04/03/2022
#### Topic: Analyzing RCT data with Precision Adjustment

In [203]:
#import Pkg; Pkg.add("DataFrames")
#import Pkg; Pkg.add("FilePaths")
#import Pkg; Pkg.add("Queryverse")
#import Pkg; Pkg.add("GLM")
#import Pkg; Pkg.add("StatsModels")
#import Pkg; Pkg.add("Combinatorics")
#import Pkg; Pkg.add("Iterators")
using GLM
using DelimitedFiles, DataFrames
using FilePaths
using Queryverse
using StatsModels, Combinatorics

In [2]:
pwd()

"C:\\Users\\Roberto Carlos\\Documents\\GitHub\\14.38_Causal_ML\\Julia_Notebooks"

In [176]:
# Loading data

mat, head = readdlm("../data/penn_jae.dat", header=true, Int16)
mat
df =DataFrame(mat, vec(head))

# describe(df)

,abdt,tg,inuidur1,inuidur2,female,black,hispanic,othrace,dep,q1
,Int16,Int16,Int16,Int16,Int16,Int16,Int16,Int16,Int16,Int16
1,10824,0,18,18,0,0,0,0,2,0
2,10635,2,7,3,0,0,0,0,0,0
3,10551,5,18,6,1,0,0,0,0,0
4,10824,0,1,1,0,0,0,0,0,0
5,10747,0,27,27,0,0,0,0,0,0
6,10544,6,7,7,0,0,0,0,0,0
7,10845,1,1,1,0,0,0,0,0,0
8,10670,3,3,3,1,0,0,0,2,0
9,10768,3,28,11,1,0,0,0,0,0


In [177]:
#dimenntions of dataframe 

a = size(df,1)
b =  size(df,2)

23

In [178]:
# Filter control group and just treatment group number 4

penn = filter(row -> row[:tg] in [4,0], df)

first(penn,20)

,abdt,tg,inuidur1,inuidur2,female,black,hispanic,othrace,dep,q1
,Int16,Int16,Int16,Int16,Int16,Int16,Int16,Int16,Int16,Int16
1,10824,0,18,18,0,0,0,0,2,0
2,10824,0,1,1,0,0,0,0,0,0
3,10747,0,27,27,0,0,0,0,0,0
4,10607,4,9,9,0,0,0,0,0,0
5,10831,0,27,27,0,0,0,0,1,0
6,10845,0,27,27,1,0,0,0,0,0
7,10831,0,9,9,1,0,0,0,1,0
8,10859,0,27,27,1,0,0,0,1,0
9,10516,0,15,15,1,0,0,0,0,0


In [179]:
# Treatment group n°4
replace!(penn.tg, 4 => 1)
penn

,abdt,tg,inuidur1,inuidur2,female,black,hispanic,othrace,dep,q1
,Int16,Int16,Int16,Int16,Int16,Int16,Int16,Int16,Int16,Int16
1,10824,0,18,18,0,0,0,0,2,0
2,10824,0,1,1,0,0,0,0,0,0
3,10747,0,27,27,0,0,0,0,0,0
4,10607,1,9,9,0,0,0,0,0,0
5,10831,0,27,27,0,0,0,0,1,0
6,10845,0,27,27,1,0,0,0,0,0
7,10831,0,9,9,1,0,0,0,1,0
8,10859,0,27,27,1,0,0,0,1,0
9,10516,0,15,15,1,0,0,0,0,0


In [180]:
describe(penn)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Int16,Float64,Int16,Int64,DataType
1,abdt,10695.4,10404,10698.0,10880,0,Int16
2,tg,0.342224,0,0.0,1,0,Int16
3,inuidur1,13.053,1,11.0,52,0,Int16
4,inuidur2,12.2812,0,10.0,52,0,Int16
5,female,0.404001,0,0.0,1,0,Int16
6,black,0.121985,0,0.0,1,0,Int16
7,hispanic,0.0325554,0,0.0,1,0,Int16
8,othrace,0.00725632,0,0.0,1,0,Int16
9,dep,0.439694,0,0.0,2,0,Int16


In [195]:
# linear regression
m = lm(@formula(tg ~ (female+black+othrace+q2+q3+q4+q5+q6+agelt35+agegt54+durable+lusd+husd)^2), penn)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

tg ~ 1 + :((female + black + othrace + q2 + q3 + q4 + q5 + q6 + agelt35 + agegt54 + durable + lusd + husd) ^ 2)

Coefficients:
──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                                                          Coef.  Std. Error      t  Pr(>|t|)     Lower 95%   Upper 95%
──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
(Intercept)                                                                                          0.326244     0.0126419  25.81    <1e-99   0.30146      0.351027
(female + black + othrace + q

In [204]:
combinations_upto(x, n) = Iterators.flatten(combinations(x, i) for i in 1:n)
expand_exp(args, deg::ConstantTerm) =
    tuple(((&)(terms...) for terms in combinations_upto(args, deg.n))...)

StatsModels.apply_schema(t::FunctionTerm{typeof(^)}, sch::StatsModels.Schema, ctx::Type) =
    apply_schema.(expand_exp(t.args_parsed...), Ref(sch), ctx)

StatsModels.apply_schema(t::FunctionTerm{typeof(^)}, sch::StatsModels.FullRank, ctx::Type) =
    apply_schema.(expand_exp(t.args_parsed...), Ref(sch), ctx)

In [205]:
# linear regression
m1 = lm(@formula(tg ~ (female+black+othrace+q2+q3+q4+q5+q6+agelt35+agegt54+durable+lusd+husd)^2), penn)
m2 = apply_schema(m1, schema(m1, husd))

LoadError: UndefVarError: husd not defined

In [ ]:
m = apply_schema(@formula(y ~  (a+b+c+d)^2), sch, StatisticalModel)